NOTE :
Test, over initial random cluster. Read markdown at the end

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import silhouette_score

train = pd.read_csv('Train_Data.csv',index_col = 0)
test = pd.read_csv('Test_Data.csv',index_col = 0)
whole = pd.concat([train,test],axis=0)
whole.reset_index(drop=True,inplace=True)
features = whole.drop(columns=['Family','Genus','Species'])
label_class = whole.iloc[:,[22,23,24]]
                   
                   

hamming_loss_vector = pd.DataFrame(index=np.arange(1,11),columns=['Hamming Loss'])
hamming_distance_vector = pd.DataFrame(index=np.arange(1,11),columns=['Hamming Distance'])
hamming_score_vector = pd.DataFrame(index=np.arange(1,11),columns=['Hamming Score'])
k_selected_vector = np.zeros(10)

for mc in np.arange(0,10):
    
    print(mc)  # to keep iteration track
                   
    sil = pd.DataFrame(index=np.arange(2,51),columns=['Silhouette_Score_Average'])
    for i in np.arange(2,51):
        kmeans = KMeans(n_clusters=i,init='random').fit(features)
        pred_clusters = kmeans.labels_
        sil.iloc[i-2] = silhouette_score(features,pred_clusters)

    max_sil = sil.max(axis=0)
    max_indx = np.argmax(sil['Silhouette_Score_Average'].to_numpy().flatten())
    k_selected = max_indx + 2
    k_selected_vector[mc] = k_selected

    kmeans = KMeans(n_clusters = k_selected, init='random').fit(features)
    cluster_groups = kmeans.labels_

    cluster_triplet_df = pd.DataFrame(index = np.arange(0,k_selected),columns=['Fam_Clus','Gen_Clus','Spec_Clus']) # has cluster label details
    for q in np.arange(0,k_selected):
        for l in np.arange(0,3):
            loc_build = []
            for p in np.arange(0,features.shape[0]):
                if cluster_groups[p] == q:
                    loc_build.append(label_class.iloc[p,l])

            build = np.array(loc_build)
            build_counts = np.bincount(build)
            highest_freq = np.argmax(build_counts)
            cluster_triplet_df.iloc[q,l] = highest_freq


    cluster_predictions_label_class = pd.DataFrame(index = np.arange(0,features.shape[0]),columns = ['Family','Genus','Species'])

    for i in np.arange(0,features.shape[0]):
        cluster_predictions_label_class.iloc[i,0] = cluster_triplet_df.iloc[cluster_groups[i],0]
        cluster_predictions_label_class.iloc[i,1] = cluster_triplet_df.iloc[cluster_groups[i],1]
        cluster_predictions_label_class.iloc[i,2] = cluster_triplet_df.iloc[cluster_groups[i],2]


    mis = 0
    for i in np.arange(0,features.shape[0]):
        for j in np.arange(0,3):
            if cluster_predictions_label_class.iloc[i,j] != label_class.iloc[i,j]:
                mis = mis + 1


    hamming_loss = (mis/(3*features.shape[0]))*100
    
    hamming_loss_vector.iloc[mc,0] = hamming_loss               

    hamming_distance = mis/features.shape[0]
    
    hamming_distance_vector.iloc[mc,0] = hamming_distance


    hs = 0
    for i in np.arange(0,features.shape[0]):

        sam_loc = 0
        for j in np.arange(0,3):

            if cluster_predictions_label_class.iloc[i,j] == label_class.iloc[i,j]:  # note the equality here, true calculated
                sam_loc = sam_loc + 1

        pred_loc = cluster_predictions_label_class.iloc[i,:].to_numpy().flatten()
        true_loc = label_class.iloc[i,:].to_numpy().flatten()

        uni_rep_strng = np.concatenate((pred_loc,true_loc)).flatten()  # union, absolute, with repetitions

        uni_bin = np.bincount(uni_rep_strng.astype(int))

        for m in np.arange(0,uni_bin.size):
            if uni_bin[m] > 1:
                uni_bin[m] = 1

        dist_cnt = np.sum(uni_bin)    # count of total distinct elements

        hs_loc = sam_loc/dist_cnt

        hs = hs + hs_loc          # running sum of HS


    hamming_score = (hs/features.shape[0])*100
    
    hamming_score_vector.iloc[mc,0] = hamming_score
                   
            

print('The hamming loss vector for all iterations, in %, is : \n',hamming_loss_vector)

print('The hamming distance vector for all iterations is : \n',hamming_distance_vector)
                   
print('The hamming score/accuracy score vector for all iterations, in %, is : \n',hamming_score_vector)         

print('k-values are : \n',k_selected_vector)

0
1
2
3
4
5
6
7
8
9
The hamming loss vector for all iterations, in %, is : 
    Hamming Loss
1       22.2469
2       22.2423
3       24.5263
4       22.2423
5       22.2423
6       22.2423
7       24.5124
8       22.2423
9       22.1635
10      22.2423
The hamming distance vector for all iterations is : 
    Hamming Distance
1          0.667408
2          0.667269
3          0.735789
4          0.667269
5          0.667269
6          0.667269
7          0.735372
8          0.667269
9          0.664906
10         0.667269
The hamming score/accuracy score vector for all iterations, in %, is : 
    Hamming Score
1        80.6139
2        80.6208
3        76.4153
4        80.6208
5        80.6208
6        80.6208
7        76.4292
8        80.6208
9        80.7396
10       80.6208
k-values are : 
 [4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]


In [4]:
avg_hamming_loss = hamming_loss_vector.mean(axis=0)
avg_hamming_distance = hamming_distance_vector.mean(axis=0)
avg_hamming_score = hamming_score_vector.mean(axis=0)

print('The average hamming loss over 10 iterations is in % : \n',avg_hamming_loss)
print('The average hamming distance over 10 iterations is : \n',avg_hamming_distance)
print('The average hamming score/accuracy over 10 iterations is in %: \n',avg_hamming_score)

The average hamming loss over 10 iterations is in % : 
 Hamming Loss    22.690294
dtype: float64
The average hamming distance over 10 iterations is : 
 Hamming Distance    0.680709
dtype: float64
The average hamming score/accuracy over 10 iterations is in %: 
 Hamming Score    79.792286
dtype: float64


This is just a test Monte Carlo case
Difference between this and the normal Monte Carlo case is as follows :

Here, K-Means, both for selecting optimal 'k' and for final clustering over that optimal 'k', initializes its initial cluster randomly, specified by init='random'

This, as specified in the documentation, does take somewhat longer time to converge, but still has been tested here.

As such, due to run time constraints, we only performed it with this randomized cluster initiation, for 10 times.

As such, there doesn't seem to be much, if at all any difference from the previous cases.

Thus we can conclude that the previous case, which initialized with an optimal initial cluster arrangement (chosen internally by k-means) did, possibly converge to the global optimum.

The run time for 10 iterations of this random initialized approach was almost half of that it took for 50 iterations with the optimal initial vector setup by k-means in scikit.

As such, if run time hadn't been so large, we would've run it also for 50 iterations just like in the previous case

But still, this result, somewhat indicates that,

k=4
HL = 22 %
HD = 0.68
HS/Accuracy = 80 %

are obtained with the clustering done, reaching its global optimum, quite sufficiently, which is what monte carlo wanted us to ensure, that the picked k, optimal, should also cluster such that we reach not a local optimum but a global one.

Running this randomized version, 50 or say 100 times, could help us get a better picture, however, of whether our original and current setup has reached the global optimum or not